In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
#import Libraries
import findspark
findspark.init('./spark-2.1.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession 
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
import pyspark.sql.functions as F

In [3]:
# sc = SparkContext()
conf = SparkConf().setAppName("pySparkMerge").setMaster("local")
sc = SparkContext.getOrCreate(conf=conf)

In [4]:
##CREATING "SPARKSESSION" from existing "SPARKCONTEXT"
sqlContext = SQLContext(sc)
spark = sqlContext.sparkSession

In [5]:
df0 = spark.read.csv('./KaggleV2-chart.csv/', header=True)
df00 = spark.read.csv('./KaggleV2-chart.csv/', header=True)

In [6]:
print((df0.count(), len(df0.columns)))
print( df0.printSchema() )
## CHECK ANY MISSING VALUSE
### Get count of both null and missing values in pyspark
from pyspark.sql.functions import isnan, when, count, col
print( df0.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df0.columns]).show() )

(110527, 14)
root
 |-- PatientId: string (nullable = true)
 |-- Scholarship: string (nullable = true)
 |-- Hipertension: string (nullable = true)
 |-- Diabetes: string (nullable = true)
 |-- Alcoholism: string (nullable = true)
 |-- Handcap: string (nullable = true)
 |-- AppointmentID: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- ScheduledDay: string (nullable = true)
 |-- AppointmentDay: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Neighbourhood: string (nullable = true)
 |-- SMS_received: string (nullable = true)
 |-- No-show: string (nullable = true)

None
+---------+-----------+------------+--------+----------+-------+-------------+------+------------+--------------+---+-------------+------------+-------+
|PatientId|Scholarship|Hipertension|Diabetes|Alcoholism|Handcap|AppointmentID|Gender|ScheduledDay|AppointmentDay|Age|Neighbourhood|SMS_received|No-show|
+---------+-----------+------------+--------+----------+-------+-------------+------+

In [7]:
# df0.select('SMS_received').show()

In [8]:
# df0.printSchema()
df0.show(2)
from pyspark.sql.types import IntegerType, FloatType
df0 = df0.withColumn("PatientId", df0["PatientId"].cast(FloatType()))
df0 = df0.withColumn("Scholarship", df0["Scholarship"].cast(IntegerType()))
df0 = df0.withColumn("Hipertension", df0["Hipertension"].cast(IntegerType()))
df0 = df0.withColumn("Diabetes", df0["Diabetes"].cast(IntegerType()))
df0 = df0.withColumn("Alcoholism", df0["Alcoholism"].cast(IntegerType()))
df0 = df0.withColumn("Handcap", df0["Handcap"].cast(IntegerType()))
df0 = df0.withColumn("AppointmentID", df0["AppointmentID"].cast(IntegerType()))
df0 = df0.withColumn("Age", df0["Age"].cast(IntegerType()))
df0 = df0.withColumn("SMS_received", df0["SMS_received"].cast(FloatType()))
df0.printSchema()

+-----------+-----------+------------+--------+----------+-------+-------------+------+--------------------+--------------------+---+---------------+------------+-------+
|  PatientId|Scholarship|Hipertension|Diabetes|Alcoholism|Handcap|AppointmentID|Gender|        ScheduledDay|      AppointmentDay|Age|  Neighbourhood|SMS_received|No-show|
+-----------+-----------+------------+--------+----------+-------+-------------+------+--------------------+--------------------+---+---------------+------------+-------+
|2.98725E+13|          0|           1|       0|         0|      0|      5642903|     F|2016-04-29T18:38:08Z|2016-04-29T00:00:00Z| 62|JARDIM DA PENHA|           0|     No|
|5.58998E+14|          0|           0|       0|         0|      0|      5642503|     M|2016-04-29T16:08:27Z|2016-04-29T00:00:00Z| 56|JARDIM DA PENHA|           0|     No|
+-----------+-----------+------------+--------+----------+-------+-------------+------+--------------------+--------------------+---+------------

In [9]:
df0.dtypes
dict(df0.dtypes)['Scholarship']

[('PatientId', 'float'),
 ('Scholarship', 'int'),
 ('Hipertension', 'int'),
 ('Diabetes', 'int'),
 ('Alcoholism', 'int'),
 ('Handcap', 'int'),
 ('AppointmentID', 'int'),
 ('Gender', 'string'),
 ('ScheduledDay', 'string'),
 ('AppointmentDay', 'string'),
 ('Age', 'int'),
 ('Neighbourhood', 'string'),
 ('SMS_received', 'float'),
 ('No-show', 'string')]

'int'

In [10]:
## Append STRING column
# for c in df0.columns:
#     if( (dict(df0.dtypes)[c] == 'int') | (dict(df0.dtypes)[c] == 'float')):
#         print(c)
omit = []
for c in df0.columns:
    if( (dict(df0.dtypes)[c] == 'string')):
        omit.append(c)
# omit.append('PatientId')
omit

['Gender', 'ScheduledDay', 'AppointmentDay', 'Neighbourhood', 'No-show']

In [11]:
### Remove the null value from SMS_received
# df0.withColumn('SMS_received',F.when(F.isnan(F.col('SMS_received'))|F.col('SMS_received').isNull(),0).otherwise(F.col('SMS_received'))

# df0 = df0.withColumn('SMS_received',F.when(F.col('SMS_received').isNull(),0).otherwise(F.col('SMS_received')))
# when(isnan(c) | col(c).isNull()

In [12]:
# from pyspark.sql.functions import col, avg
# def fill_with_mean(df, exclude=set()): 
#     for c in df0.columns:
#         if( (dict(df0.dtypes)[c] == 'int') | (dict(df0.dtypes)[c] == 'float')):
#             stats = df0.agg(*(
#                 avg(c).alias(c) 
#                 ))        
#     return df0.na.fill(stats.first().asDict())

In [13]:
### WORKING ONE , I made it applied mean only for SMS_received
from pyspark.sql.functions import col, avg
def fill_with_mean(df, exclude=set()): 
#     print(exclude)
    stats = df.agg(*(
        avg(c).alias(c) for c in df.columns if c not in exclude
    ))
    return df.na.fill(stats.first().asDict())

# fill_with_mean(df_data, ["id", "date"])
# df0 = fill_with_mean(df0, omit)

In [14]:
df0 = fill_with_mean(df0, omit)

## df1 = df0.rdd.map(lambda x: x.na.fill(meanOrMode(x)))
# df1.count()

## summary4 = df0.isnull().any()
# df0.show(3)

In [15]:
# df0.select('PatientId').show(3)

# df0.withColumn('PatientId', df00.PatientId).show(3)

In [16]:
## CHECK ANY MISSING VALUSE
### Get count of both null and missing values in pyspark
from pyspark.sql.functions import isnan, when, count, col
print( df0.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df0.columns]).show() )

+---------+-----------+------------+--------+----------+-------+-------------+------+------------+--------------+---+-------------+------------+-------+
|PatientId|Scholarship|Hipertension|Diabetes|Alcoholism|Handcap|AppointmentID|Gender|ScheduledDay|AppointmentDay|Age|Neighbourhood|SMS_received|No-show|
+---------+-----------+------------+--------+----------+-------+-------------+------+------------+--------------+---+-------------+------------+-------+
|        0|          0|           0|       0|         0|      0|            0|     0|           0|             0|  0|            0|           0|      0|
+---------+-----------+------------+--------+----------+-------+-------------+------+------------+--------------+---+-------------+------------+-------+

None


In [17]:
# Check all appointment ids are unique -- value_counts() 
print(df0.groupBy('AppointmentID').count().orderBy('count').show())

+-------------+-----+
|AppointmentID|count|
+-------------+-----+
|      5642643|    1|
|      5637496|    1|
|      5629170|    1|
|      5615023|    1|
|      5638919|    1|
|      5637655|    1|
|      5637879|    1|
|      5642171|    1|
|      5548957|    1|
|      5642795|    1|
|      5640781|    1|
|      5640214|    1|
|      5638301|    1|
|      5521710|    1|
|      5522956|    1|
|      5626229|    1|
|      5630294|    1|
|      5637310|    1|
|      5683766|    1|
|      5486000|    1|
+-------------+-----+
only showing top 20 rows

None


In [18]:
# Remove negative value for Age 
df0.createOrReplaceTempView("df0_tbl")

query = '''SELECT min(Age), max(Age) FROM df0_tbl '''
print(spark.sql(query).show(5))
print(df0.describe().show())

df0 = df0.withColumn('Age', when(col('Age') != -1,col('Age')) )

# df= df.withColumn('foo', when(col('foo') != 'empty-value',col('foo)))
df0.createOrReplaceTempView("df0_tbl")

query = '''SELECT min(Age), max(Age) FROM df0_tbl '''
spark.sql(query).show(5)

+--------+--------+
|min(Age)|max(Age)|
+--------+--------+
|      -1|     115|
+--------+--------+

None
+-------+--------------------+-------------------+-------------------+-------------------+--------------------+--------------------+-----------------+------+--------------------+--------------------+------------------+-------------+-------------------+-------+
|summary|           PatientId|        Scholarship|       Hipertension|           Diabetes|          Alcoholism|             Handcap|    AppointmentID|Gender|        ScheduledDay|      AppointmentDay|               Age|Neighbourhood|       SMS_received|No-show|
+-------+--------------------+-------------------+-------------------+-------------------+--------------------+--------------------+-----------------+------+--------------------+--------------------+------------------+-------------+-------------------+-------+
|  count|              110527|             110527|             110527|             110527|              110527|

In [19]:
# df0.columns
# df0['Gender'] = df0['Gender'].map({'M':0,'F':1})
# df0['No-show'] = df0['No-show'].map({'Yes':1, 'No':0})

In [20]:
## TRANSFORM FEATURES - ENCODE DATA

# Assign target and gender feature as binaries
# For 'No-show': Yes = 1, No = 0
# For 'Gender: F = 1, M = 0
from functools import reduce

cols = ['Gender']
df0 = reduce(lambda df, c: df.withColumn(c, F.when(df[c] == 'M', 0).otherwise(1)), cols, df0)

cols = ['No-show']
df0 = reduce(lambda df, c: df.withColumn(c, F.when(df[c] == 'No', 0).otherwise(1)), cols, df0)

df0.show(3)

+-------------+-----------+------------+--------+----------+-------+-------------+------+--------------------+--------------------+---+---------------+------------+-------+
|    PatientId|Scholarship|Hipertension|Diabetes|Alcoholism|Handcap|AppointmentID|Gender|        ScheduledDay|      AppointmentDay|Age|  Neighbourhood|SMS_received|No-show|
+-------------+-----------+------------+--------+----------+-------+-------------+------+--------------------+--------------------+---+---------------+------------+-------+
|   2.98725E13|          0|           1|       0|         0|      0|      5642903|     1|2016-04-29T18:38:08Z|2016-04-29T00:00:00Z| 62|JARDIM DA PENHA|         0.0|      0|
|5.58998013E14|          0|           0|       0|         0|      0|      5642503|     0|2016-04-29T16:08:27Z|2016-04-29T00:00:00Z| 56|JARDIM DA PENHA|         0.0|      0|
|4.26296004E12|          0|           0|       0|         0|      0|      5642549|     1|2016-04-29T16:19:04Z|2016-04-29T00:00:00Z| 62|

In [21]:
# pyspark.sql.functions.split(df0['ScheduledDay'], 'T').collect().show(2)
split_col = pyspark.sql.functions.split(df0['ScheduledDay'], 'T')
df0 = df0.withColumn('ScheduledDay_Day', split_col.getItem(0))
df0 = df0.withColumn('ScheduledDay_Time', split_col.getItem(1))

split_col = pyspark.sql.functions.split(df0['AppointmentDay'], 'T')
df0 = df0.withColumn('AppointmentDay_Day', split_col.getItem(0))
df0 = df0.withColumn('AppointmentDay_Time', split_col.getItem(1))

In [22]:
df0.show(2)

+-------------+-----------+------------+--------+----------+-------+-------------+------+--------------------+--------------------+---+---------------+------------+-------+----------------+-----------------+------------------+-------------------+
|    PatientId|Scholarship|Hipertension|Diabetes|Alcoholism|Handcap|AppointmentID|Gender|        ScheduledDay|      AppointmentDay|Age|  Neighbourhood|SMS_received|No-show|ScheduledDay_Day|ScheduledDay_Time|AppointmentDay_Day|AppointmentDay_Time|
+-------------+-----------+------------+--------+----------+-------+-------------+------+--------------------+--------------------+---+---------------+------------+-------+----------------+-----------------+------------------+-------------------+
|   2.98725E13|          0|           1|       0|         0|      0|      5642903|     1|2016-04-29T18:38:08Z|2016-04-29T00:00:00Z| 62|JARDIM DA PENHA|         0.0|      0|      2016-04-29|        18:38:08Z|        2016-04-29|          00:00:00Z|
|5.58998013E

In [23]:
# No 'time stamp' in 'AppointmentDay' features
print(df0.groupBy('AppointmentDay_Time').count().orderBy('count').show())

+-------------------+------+
|AppointmentDay_Time| count|
+-------------------+------+
|          00:00:00Z|110527|
+-------------------+------+

None


In [24]:
df0.show(2)

+-------------+-----------+------------+--------+----------+-------+-------------+------+--------------------+--------------------+---+---------------+------------+-------+----------------+-----------------+------------------+-------------------+
|    PatientId|Scholarship|Hipertension|Diabetes|Alcoholism|Handcap|AppointmentID|Gender|        ScheduledDay|      AppointmentDay|Age|  Neighbourhood|SMS_received|No-show|ScheduledDay_Day|ScheduledDay_Time|AppointmentDay_Day|AppointmentDay_Time|
+-------------+-----------+------------+--------+----------+-------+-------------+------+--------------------+--------------------+---+---------------+------------+-------+----------------+-----------------+------------------+-------------------+
|   2.98725E13|          0|           1|       0|         0|      0|      5642903|     1|2016-04-29T18:38:08Z|2016-04-29T00:00:00Z| 62|JARDIM DA PENHA|         0.0|      0|      2016-04-29|        18:38:08Z|        2016-04-29|          00:00:00Z|
|5.58998013E

In [25]:
# Remove Z trailed at the value end
# from pyspark.sql.functions import substring, length
# df1 = df0.withColumn("ScheduledDay_Time",substring(col("ScheduledDay_Time"),0,length(col("ScheduledDay_Time"))-1))
df0 = df0.withColumn("ScheduledDay_Time", df0["ScheduledDay_Time"].substr(0,8))

In [26]:
# Create feature for hour of the day from 'scheduledDay'
df0 = df0.withColumn('ScheduledDay_Hours', df0['ScheduledDay_Time'].substr(0,2))

In [27]:
df0.show(2)

+-------------+-----------+------------+--------+----------+-------+-------------+------+--------------------+--------------------+---+---------------+------------+-------+----------------+-----------------+------------------+-------------------+------------------+
|    PatientId|Scholarship|Hipertension|Diabetes|Alcoholism|Handcap|AppointmentID|Gender|        ScheduledDay|      AppointmentDay|Age|  Neighbourhood|SMS_received|No-show|ScheduledDay_Day|ScheduledDay_Time|AppointmentDay_Day|AppointmentDay_Time|ScheduledDay_Hours|
+-------------+-----------+------------+--------+----------+-------+-------------+------+--------------------+--------------------+---+---------------+------------+-------+----------------+-----------------+------------------+-------------------+------------------+
|   2.98725E13|          0|           1|       0|         0|      0|      5642903|     1|2016-04-29T18:38:08Z|2016-04-29T00:00:00Z| 62|JARDIM DA PENHA|         0.0|      0|      2016-04-29|         18:3

In [28]:
# Convert to 'datetime' features dateType conversion
from pyspark.sql.types import DateType
df0 = df0.withColumn("ScheduledDay_Day",df0['ScheduledDay_Day'].cast(DateType()))
df0 = df0.withColumn("AppointmentDay_Day",df0['AppointmentDay_Day'].cast(DateType()))
df0.printSchema()

root
 |-- PatientId: float (nullable = false)
 |-- Scholarship: integer (nullable = true)
 |-- Hipertension: integer (nullable = true)
 |-- Diabetes: integer (nullable = true)
 |-- Alcoholism: integer (nullable = true)
 |-- Handcap: integer (nullable = true)
 |-- AppointmentID: integer (nullable = true)
 |-- Gender: integer (nullable = false)
 |-- ScheduledDay: string (nullable = true)
 |-- AppointmentDay: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Neighbourhood: string (nullable = true)
 |-- SMS_received: float (nullable = false)
 |-- No-show: integer (nullable = false)
 |-- ScheduledDay_Day: date (nullable = true)
 |-- ScheduledDay_Time: string (nullable = true)
 |-- AppointmentDay_Day: date (nullable = true)
 |-- AppointmentDay_Time: string (nullable = true)
 |-- ScheduledDay_Hours: string (nullable = true)



In [29]:
# Create feature for date for scheduled date 'DD'
df0 = df0.withColumn('ScheduledDay_Date', df0['ScheduledDay_Day'].substr(9,10))
df0 = df0.withColumn('AppointmentDay_Date', df0['AppointmentDay_Day'].substr(9,10))
df0.show(2) 

+-------------+-----------+------------+--------+----------+-------+-------------+------+--------------------+--------------------+---+---------------+------------+-------+----------------+-----------------+------------------+-------------------+------------------+-----------------+-------------------+
|    PatientId|Scholarship|Hipertension|Diabetes|Alcoholism|Handcap|AppointmentID|Gender|        ScheduledDay|      AppointmentDay|Age|  Neighbourhood|SMS_received|No-show|ScheduledDay_Day|ScheduledDay_Time|AppointmentDay_Day|AppointmentDay_Time|ScheduledDay_Hours|ScheduledDay_Date|AppointmentDay_Date|
+-------------+-----------+------------+--------+----------+-------+-------------+------+--------------------+--------------------+---+---------------+------------+-------+----------------+-----------------+------------------+-------------------+------------------+-----------------+-------------------+
|   2.98725E13|          0|           1|       0|         0|      0|      5642903|     1

In [30]:
# # Calculate time difference from 'ScheduleDay' to 'AppointmentDay'  
# Extract only the days - output will be a rouned number 
# df0['day_diff'] = df0['AppointmentDay_Day'] - df0['ScheduledDay_Day']
# df0.info()
from pyspark.sql.functions import datediff, to_date, lit, unix_timestamp
df0 = df0.withColumn("day_diff",datediff('AppointmentDay_Day','ScheduledDay_Day'))
df0.show(3)

+-------------+-----------+------------+--------+----------+-------+-------------+------+--------------------+--------------------+---+---------------+------------+-------+----------------+-----------------+------------------+-------------------+------------------+-----------------+-------------------+--------+
|    PatientId|Scholarship|Hipertension|Diabetes|Alcoholism|Handcap|AppointmentID|Gender|        ScheduledDay|      AppointmentDay|Age|  Neighbourhood|SMS_received|No-show|ScheduledDay_Day|ScheduledDay_Time|AppointmentDay_Day|AppointmentDay_Time|ScheduledDay_Hours|ScheduledDay_Date|AppointmentDay_Date|day_diff|
+-------------+-----------+------------+--------+----------+-------+-------------+------+--------------------+--------------------+---+---------------+------------+-------+----------------+-----------------+------------------+-------------------+------------------+-----------------+-------------------+--------+
|   2.98725E13|          0|           1|       0|         0| 

In [31]:
# outliers - negative date differences 
print(df0.describe().select("day_diff").show()) 
print(df0.select('day_diff').show())
# df.select("col").show()

+------------------+
|          day_diff|
+------------------+
|            110527|
|10.183701719941734|
|15.254996224106186|
|                -6|
|               179|
+------------------+

None
+--------+
|day_diff|
+--------+
|       0|
|       0|
|       0|
|       0|
|       0|
|       2|
|       2|
|       2|
|       0|
|       2|
|       2|
|       3|
|       1|
|       1|
|       1|
|       3|
|       1|
|       1|
|       3|
|       0|
+--------+
only showing top 20 rows

None


In [32]:
# df0.select('day_diff')
df0.where(df0.day_diff < 0).show()

+-------------+-----------+------------+--------+----------+-------+-------------+------+--------------------+--------------------+---+-------------+------------+-------+----------------+-----------------+------------------+-------------------+------------------+-----------------+-------------------+--------+
|    PatientId|Scholarship|Hipertension|Diabetes|Alcoholism|Handcap|AppointmentID|Gender|        ScheduledDay|      AppointmentDay|Age|Neighbourhood|SMS_received|No-show|ScheduledDay_Day|ScheduledDay_Time|AppointmentDay_Day|AppointmentDay_Time|ScheduledDay_Hours|ScheduledDay_Date|AppointmentDay_Date|day_diff|
+-------------+-----------+------------+--------+----------+-------+-------------+------+--------------------+--------------------+---+-------------+------------+-------+----------------+-----------------+------------------+-------------------+------------------+-----------------+-------------------+--------+
|   7.83927E12|          0|           0|       0|         0|      1

In [33]:
# Replace with 'nan' so that they will be dropped later
df0 = df0.withColumn('day_diff',F.when(df0['day_diff']<0,None).otherwise(F.col('day_diff')))

# Check any negative value again
df0.where(df0.day_diff < 0).show()

+---------+-----------+------------+--------+----------+-------+-------------+------+------------+--------------+---+-------------+------------+-------+----------------+-----------------+------------------+-------------------+------------------+-----------------+-------------------+--------+
|PatientId|Scholarship|Hipertension|Diabetes|Alcoholism|Handcap|AppointmentID|Gender|ScheduledDay|AppointmentDay|Age|Neighbourhood|SMS_received|No-show|ScheduledDay_Day|ScheduledDay_Time|AppointmentDay_Day|AppointmentDay_Time|ScheduledDay_Hours|ScheduledDay_Date|AppointmentDay_Date|day_diff|
+---------+-----------+------------+--------+----------+-------+-------------+------+------------+--------------+---+-------------+------------+-------+----------------+-----------------+------------------+-------------------+------------------+-----------------+-------------------+--------+
+---------+-----------+------------+--------+----------+-------+-------------+------+------------+--------------+---+----

In [34]:
# df0.columns
# .select('df00.PatientId','df0.Scholarship','df0.Hipertension','df0.Diabetes','df0.Alcoholism','df0.Handcap','df0.AppointmentID','df0.Gender','df0.ScheduledDay','df0.AppointmentDay','df0.Age','df0.Neighbourhood','df0.SMS_received','df0.No-show','df0.ScheduledDay_Day','df0.ScheduledDay_Time','df0.AppointmentDay_Day','df0.AppointmentDay_Time','df0.ScheduledDay_Hours','df0.ScheduledDay_Date','df0.AppointmentDay_Date','df0.day_diff')

In [35]:
# df00.select('PatientId').collect()
# df0.withColumn('pid', df00.select('PatientId').collect())

# df0 = df0.alias('df0').join(df00.alias('df00'), df0.AppointmentID == df00.AppointmentID ).select('df00.PatientId','df0.Scholarship','df0.Hipertension','df0.Diabetes','df0.Alcoholism','df0.Handcap','df0.AppointmentID','df0.Gender','df0.ScheduledDay','df0.AppointmentDay','df0.Age','df0.Neighbourhood','df0.SMS_received','df0.No-show','df0.ScheduledDay_Day','df0.ScheduledDay_Time','df0.AppointmentDay_Day','df0.AppointmentDay_Time','df0.ScheduledDay_Hours','df0.ScheduledDay_Date','df0.AppointmentDay_Date','df0.day_diff')

In [36]:
# Transform Feature - 'precedent_appointments' (new)
# Create a new feature to tell number of precedent appointments 
# Keep only the last entry for the patient, hence need to minus 1 appointment from the total 

# df0.groupby('PatientId').count().show()
import pyspark.sql.functions as f
appointments = df0.groupBy('PatientId').count().select('PatientId', f.col('count').alias('precedent_appointments'))
appointments.columns
appointments.select('precedent_appointments').show(3)
appointments = appointments.withColumn('precedent_appointments', appointments['precedent_appointments']-1)

['PatientId', 'precedent_appointments']

+----------------------+
|precedent_appointments|
+----------------------+
|                     1|
|                     1|
|                     2|
+----------------------+
only showing top 3 rows



In [37]:
appointments.show(3)
df0 = df0.join(appointments, how = 'left', on = 'PatientId')

+------------+----------------------+
|   PatientId|precedent_appointments|
+------------+----------------------+
|8.4574399E13|                     0|
|4.9823802E12|                     0|
|8.6471298E13|                     1|
+------------+----------------------+
only showing top 3 rows



In [38]:
# Transform Feature - 'precedent_no_shows' (new)
# Create a new feature to tell number of precedent no-show records 
# Repeat the step for no-shows

# df0.groupBy('PatientId').count().select('PatientId', f.col('count').alias('precedent_appointments'))
from pyspark.sql.functions import sum as _sum
no_shows = df0.groupBy('PatientId').agg(_sum('No-show').alias('precedent_no_shows'))
no_shows = no_shows.withColumn('precedent_no_shows', no_shows['precedent_no_shows']-1)
no_shows.columns
no_shows.show(3)

['PatientId', 'precedent_no_shows']

+------------+------------------+
|   PatientId|precedent_no_shows|
+------------+------------------+
|8.4574399E13|                -1|
|4.9823802E12|                -1|
|8.6471298E13|                -1|
+------------+------------------+
only showing top 3 rows



In [39]:
df0 = df0.join(no_shows, how = 'left', on = 'PatientId')
df0 = df0.withColumn("precedent_no_shows",when(df0["precedent_no_shows"] == -1,0).otherwise(df0["precedent_no_shows"]))

In [40]:
df0.columns

['PatientId',
 'Scholarship',
 'Hipertension',
 'Diabetes',
 'Alcoholism',
 'Handcap',
 'AppointmentID',
 'Gender',
 'ScheduledDay',
 'AppointmentDay',
 'Age',
 'Neighbourhood',
 'SMS_received',
 'No-show',
 'ScheduledDay_Day',
 'ScheduledDay_Time',
 'AppointmentDay_Day',
 'AppointmentDay_Time',
 'ScheduledDay_Hours',
 'ScheduledDay_Date',
 'AppointmentDay_Date',
 'day_diff',
 'precedent_appointments',
 'precedent_no_shows']

In [41]:
# Too many levels in 'Neighbourhood' variables
# Difficult to append additional info about the neigbourhoods
print(df0.groupBy('Neighbourhood').count().orderBy('count').show())

+--------------------+-----+
|       Neighbourhood|count|
+--------------------+-----+
|   PARQUE INDUSTRIAL|    1|
|ILHAS OCE?NICAS D...|    2|
|           AEROPORTO|    8|
|       ILHA DO FRADE|   10|
|         ILHA DO BOI|   35|
|   PONTAL DE CAMBURI|   69|
|   MORADA DE CAMBURI|   96|
|            NAZARETH|  135|
|    SEGURAN?A DO LAR|  145|
|       UNIVERSIT?RIO|  152|
|               HORTO|  175|
|        SANTA HELENA|  178|
|      ENSEADA DO SU?|  235|
|           FRADINHOS|  258|
|     ANT?NIO HON?RIO|  271|
| ARIOVALDO FAVALESSA|  282|
|          DE LOURDES|  305|
|             COMDUSA|  310|
|           BOA VISTA|  312|
|      M?RIO CYPRESTE|  371|
+--------------------+-----+
only showing top 20 rows

None


In [42]:
# Remove Duplicate PatientIds, Keep Latest Appointment Entry Only
df1 = df0
# df1.sort_values('AppointmentDay', inplace = True)
# df1.drop_duplicates(subset='PatientId', keep = 'last', inplace = True)
# print(df1)
# df1.info()

In [43]:
print(df1.count())
df1 = df1.drop_duplicates(subset=['PatientId'])

110527


In [44]:
df1 = df1.select('Gender', 'Age', 'Scholarship', 'Hipertension', 'Diabetes',
       'Alcoholism', 'Handcap', 'SMS_received', 'No-show',
       'ScheduledDay_Hours', 'ScheduledDay_Date', 'AppointmentDay_Date',
       'day_diff', 'precedent_appointments', 
       'precedent_no_shows')

In [45]:
df1.count()
df0.count()

61744

110527

In [46]:
# Remove NA values for variables (age & day_diff) 
from pyspark.sql.functions import isnan, when, count, col
df1 = df1.withColumn('age',F.when(F.col('age').isNull(),0).otherwise(F.col('age')))
df1.where(col('age').isNull()).count()

df1 = df1.withColumn('day_diff',F.when(F.col('day_diff').isNull(),0).otherwise(F.col('day_diff')))
df1.where(col('day_diff').isNull()).count()

0

0

In [47]:
# Show NA values (if any)
print( df1.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df1.columns]).show() )

+------+---+-----------+------------+--------+----------+-------+------------+-------+------------------+-----------------+-------------------+--------+----------------------+------------------+
|Gender|age|Scholarship|Hipertension|Diabetes|Alcoholism|Handcap|SMS_received|No-show|ScheduledDay_Hours|ScheduledDay_Date|AppointmentDay_Date|day_diff|precedent_appointments|precedent_no_shows|
+------+---+-----------+------------+--------+----------+-------+------------+-------+------------------+-----------------+-------------------+--------+----------------------+------------------+
|     0|  0|          0|           0|       0|         0|      0|           0|      0|                 0|                0|                  0|       0|                     0|                 0|
+------+---+-----------+------------+--------+----------+-------+------------+-------+------------------+-----------------+-------------------+--------+----------------------+------------------+

None


In [48]:
df1.printSchema()

root
 |-- Gender: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- Scholarship: integer (nullable = true)
 |-- Hipertension: integer (nullable = true)
 |-- Diabetes: integer (nullable = true)
 |-- Alcoholism: integer (nullable = true)
 |-- Handcap: integer (nullable = true)
 |-- SMS_received: float (nullable = true)
 |-- No-show: integer (nullable = true)
 |-- ScheduledDay_Hours: string (nullable = true)
 |-- ScheduledDay_Date: string (nullable = true)
 |-- AppointmentDay_Date: string (nullable = true)
 |-- day_diff: integer (nullable = true)
 |-- precedent_appointments: long (nullable = true)
 |-- precedent_no_shows: long (nullable = true)



In [49]:
from pyspark.sql.types import IntegerType
df1 = df1.withColumn("SMS_received", df1["SMS_received"].cast(IntegerType()))
df1 = df1.withColumn("ScheduledDay_Hours", df1["ScheduledDay_Hours"].cast(IntegerType()))
df1 = df1.withColumn("ScheduledDay_Date", df1["ScheduledDay_Date"].cast(IntegerType()))
df1 = df1.withColumn("AppointmentDay_Date", df1["AppointmentDay_Date"].cast(IntegerType()))
df1 = df1.withColumn("precedent_appointments", df1["precedent_appointments"].cast(IntegerType()))
df1 = df1.withColumn("precedent_no_shows", df1["precedent_no_shows"].cast(IntegerType()))
df1.printSchema()

root
 |-- Gender: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- Scholarship: integer (nullable = true)
 |-- Hipertension: integer (nullable = true)
 |-- Diabetes: integer (nullable = true)
 |-- Alcoholism: integer (nullable = true)
 |-- Handcap: integer (nullable = true)
 |-- SMS_received: integer (nullable = true)
 |-- No-show: integer (nullable = true)
 |-- ScheduledDay_Hours: integer (nullable = true)
 |-- ScheduledDay_Date: integer (nullable = true)
 |-- AppointmentDay_Date: integer (nullable = true)
 |-- day_diff: integer (nullable = true)
 |-- precedent_appointments: integer (nullable = true)
 |-- precedent_no_shows: integer (nullable = true)



In [50]:
## Align the column naming convention
df1 = df1.withColumnRenamed('Gender', 'gender')
df1 = df1.withColumnRenamed('Age', 'age')
df1 = df1.withColumnRenamed('Scholarship', 'scholarship')
df1 = df1.withColumnRenamed('Hipertension', 'hypertension')
df1 = df1.withColumnRenamed('Diabetes', 'diabetes')
df1 = df1.withColumnRenamed('Alcoholism', 'alcoholism')
df1 = df1.withColumnRenamed('Handcap', 'handicap')
df1 = df1.withColumnRenamed('SMS_received', 'sms')
df1 = df1.withColumnRenamed('No-show', 'no_show')
df1 = df1.withColumnRenamed('ScheduledDay_Hours', 'schedule_hour')
df1 = df1.withColumnRenamed('ScheduledDay_Date', 'schedule_day')
df1 = df1.withColumnRenamed('AppointmentDay_Date', 'appointment_day')
df1 = df1.withColumnRenamed('precedent_no_shows', 'precedent_noshows')

In [51]:
df1.columns

['gender',
 'age',
 'scholarship',
 'hypertension',
 'diabetes',
 'alcoholism',
 'handicap',
 'sms',
 'no_show',
 'schedule_hour',
 'schedule_day',
 'appointment_day',
 'day_diff',
 'precedent_appointments',
 'precedent_noshows']

In [52]:
# Save the Processed File
# df1.to_pickle('df1.clean')
df1.rdd.saveAsPickleFile('./df1.clean')